# Running Molecular Dynamics Simulations in OpenMM

## Overview
**Molecular Dynamics Simulation**: simulation of molecules through time via calculations of forces on atoms and their positions based on those forces. The output is a trajectory.

## Steps for running a simulation
1. **Initialization** -- setting up imports, variables, and other system parameters to prepare for the simulation
2. **Minimization** -- calculating the energies of moving molecules slightly to obtain a local "minimum" in energy and eliminate “bad” interatomic contacts
3. **Equilibration** -- a brief MD simulation that prepares our system to our target temperature and desired equilibrium values
4. **Production** -- run the equilibration
5. **Analysis** -- after you have collected your data from the production run, you must analyze the trajectory to draw conclusions

## Our First Simulations
We will now use [OpenMM](https://openmm.org/documentation) to do a molecular dynamics simulation of the ethane and butane molecules we prepared. It’s important to note at this point that molecular dynamics simulations can be performed using a number of softwares. However, we will be running a simulation with a program called OpenMM. OpenMM has the advantage of being scriptable with Python.

---

### Initialization
Once you have OpenMM, we can use it to simulate our molecules. Start in your notebook with imports. Here are the python libraries you will need to run simulations with OpenMM.

In [1]:
import openmm as mm
from openmm import app, unit

First, we need to read in our structure and our force field. We have to tell the simulation our initial coordinates and the force field we will use. To do this, we use the PDB file we have and the force field file we prepared.

In [2]:
pdb = app.PDBFile('data/ethane.pdb')
forcefield = app.ForceField('data/ethane.gaff2.xml')

#### QUESTION
What are we doing with these two commands above?
*(Hint: let's check out some [documentation](http://docs.openmm.org/latest/userguide/application/02_running_sims.html))*

#### RESPONSE
First we must load the data from ethane.pdb which contains the atom types and the topology, which is how the atoms are bonded together. We store that data in a variable pdb. Then we load the data from ethane.gaff2.xml which contains more information about the force field we wish to use for ethane. This xml file contains the atoms, their topology, and the parameters used in the calculation of v(r), the force field. This includes parameters for bonded interactions, bond angles, torsional angles, van der walls interactions, and electrostatic interactions.

#### QUESTION
What lies in the contents of the XML file being read in `forcefield`?

#### RESPONSE

The xml file contains atom types, residues, and the parameters for bonded interactions, bond angles, torisonal angles, van der walls interactions, and electrostatic interactions. The atom types must denote an atom for each unique atom, which considers how the atom is bonded to other atoms. In ethane there is one unique carbon, and one unique hydrogen, as the carbons are equivalent and the hydrogens are all equivalent as well. The residue is a model of the molecules in the simulation which is one, and it contains the connectivity of the atoms which form the ethane molecule. Then the parameters are set for the calculation of the forcefield.

Next, we set up the system for our MD simulation. With the following command, we use the `pdb` variable to create a system. The other arguments to the function say that we are not using a cut-off and that we want to constrain bonds with hydrogens (this allows us to use a larger timestep).

In [3]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

Our simulation will be in vacuum at a temperature of 298.15 K. The Langevin integrator is what is called a stochastic integrator. This means that it mimics jostling of air or solvent through random forces. We are using a 5.0 picosecond coupling constant, which is something which controls how often the integrator adds jostling motions.

In [18]:
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

#### QUESTION
What is the `integrator` applying to our simulation?

#### RESPONSE
The integrator is serving as a proxy for the jostling motion that would be applied by air or solvent. This is necessary because the system is initalized to be in a vacuum and thus there are no solvent molecules being simulated. The integrators stochastic nature will exert change on the ethane molecules through random variation. The integrator takes as input the system temperature, the friction coefficient, and the time step.

Finally, we initialize the simulation by adding all of the pieces we have prepared:

In [19]:
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform) # ENTER THE THREE PIECES OF OUR SIMULATION HERE (hmm... I wonder where I can find this)
simulation.context.setPositions(pdb.positions)

---

### Minimization
Now, we start calculating energies. First we do an energy minimization. We start in this code block by printing the energy before minimization, doing 100 steps of an energy minimization, then printing the new energy. You should see that the energy decreases:

In [20]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

Minimizing...
Potential energy before minimization is 4.467818271065182 kJ/mol
Potential energy after minimization is 4.389967724953268 kJ/mol


You can’t see it from this code, but the atom positions have changed slightly to cause this change in energy.

#### QUESTION
What is an energy minimization? Why do we want to perform a local energy minimization at the start of our simulation?

#### RESPONSE
The energy minimzation takes the coordinates from the pdb file containing positions and calculates the potential energy among the molecules, then presumably updates the positions of each molecule to be moving down the gradient towards less potential energy. This is necessary to to at the beginning of a simulation in case the pdb positions represent a state with very high, unrealistic forces between molecules. The minimization moves the intial system to a more realistic state.

---

### Equilibration
Next, we run an equilibration. The purpose of this equilibration is to get our system to our target temperature and to get the system equilibrated and ready for our production run.

In [21]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,14.034846213609665,412.88567572831795
200,15.944912376236335,388.5507431282175
300,28.159596278419997,350.79407617666766
400,23.9520294528912,394.93212867069366
500,15.59985261631188,332.2038826503667
600,25.999676640999496,110.32820423639593
700,21.570115844409056,261.0723585511351
800,24.351522500809995,133.7906846768851
900,19.762045500392254,429.22206807823056
1000,26.91405153665985,332.1295823316184
1100,20.52274369939086,365.5866505888021
1200,22.812544039872915,242.1510425797455
1300,22.636919811118467,323.4302376205737
1400,25.35178185753454,272.2594026683726
1500,32.18577069393248,284.6804649383162
1600,17.901303322711495,237.24293435016486
1700,14.378552424757665,404.45663671410244
1800,30.059767863168634,309.1050789155913
1900,22.93406958258751,386.59972366412404
2000,28.886336838732397,219.76102578812768
2100,13.342239228830728,198.5104834847249
2200,13.533657648749337,182.60273279679996
2300,22.293

The first command in this section sets up what information OpenMM will report to us as the simulation runs. We are asking for the potential energy, and temperature every 100 timesteps. By putting `stdout` as the first argument, we are telling the program to just print the information. Note that `stdout` comes from the built in Python module `sys`. If we wanted the information in a file instead, you would put the file name.

The second command sets the initial velocities of the system to a temperature equivalent of 150 K. Then, we integrate for 2,500 steps to allow the system to equilibrate.

---

### Production
This next block of code is a longer block of simulation called the ‘production run’. We’re also added a timer to this code so we can see how long it took our simulation to run.

In [22]:
import time as time

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 250000 steps - (which is equal to 2 fs(250,000) = 500,000 fs = 500 ps)
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step(10000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
250000,500.0000000016593,14.378065665634262,202.9409878127568,0
500000,999.9999999901769,17.053142933051717,418.55256695036036,2.8e+04
750000,1499.9999999783536,12.097997078020308,332.29680251797373,2.79e+04
1000000,1999.9999999665301,30.983496808126063,207.77569112297954,2.78e+04
1250000,2499.9999999547067,29.316254526233987,293.3095917595377,2.78e+04
1500000,2999.9999999428833,21.858492133885846,344.93881211962747,2.78e+04
1750000,3499.99999993106,24.09512744619221,222.81075265149383,2.78e+04
2000000,3999.9999999192364,16.076376031470353,186.43552923010273,2.77e+04
2250000,4499.9999999992715,17.210450003614113,370.3145374086201,2.77e+04
2500000,5000.000000101135,13.09636570810093,227.74052911896055,2.74e+04
2750000,5500.000000202998,10.055229922402686,221.5629474702949,2.74e+04
3000000,6000.000000304862,19.00087542209232,103.14323592866705,2.74e+04
3250000,6500.000000406725,20.01

After executing this cell in your notebook, you should see an output which gives you the step number, simulation time, potential energy, temperature, and “speed” for steps in the simulation. The spacing of theses is set in the `simulation.reporters` step where we indicated we wanted information printed every 250,000 timesteps.

The “speed” is reported in “ns/day” or “nanoseconds/day”. This is a commonly used unit to report how quickly simulations run. It tells you how much simulation time in nanoseconds would pass for 24 hours of computation time. For example, if a simulation is running at 2 ns/day, enough timesteps would be calculated in one day to make 2 nanoseconds of simulation time. If we were using our 2 fs timestep, this would mean that the computer calculated 1,000,000 timesteps over 24 hours.

---

## Your Turn
Make a copy of the code you wrote to run your ethane simulation and modify it to:
1. Read in the files `butane.gaff2.xml` and `butane.pdb`
2. Carry out a 10 ps MD simulation to bring the butane molecule to an equilibrium temperature of 298 K in which output is printed every 0.5 ps (Leave the minimization portion beforehand unchanged.)
3. Carry out a 40 ns MD simulation at 298 K in which output is printed every 1 ns and structures are (still) saved every 0.2 ps into a file called `butane_sim.dcd`.

In [30]:
### your code here!!!

pdb = app.PDBFile('data/butane.pdb')
forcefield = app.ForceField('data/butane.gaff2.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform) # ENTER THE THREE PIECES OF OUR SIMULATION HERE (hmm... I wonder where I can find this)
simulation.context.setPositions(pdb.positions)

print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

from sys import stdout

print('Equilibrating...')

#print data to stdout every 250 steps, 0.5 ps
simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(298*unit.kelvin)

#run for 5000 steps (2 fs * 5000 = 10ps)
simulation.step(5000)

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 500,000 steps - (which is equal to 2 fs(500,000) = 1,000,000 fs = 1 ns)
simulation.reporters.append(app.StateDataReporter(stdout, 500000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('butane_sim.dcd', 100))

# run the simulation for 40 ns (= 2.0x10^7 steps - 20 ns )
simulation.step(20000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')




Minimizing...
Potential energy before minimization is 5.797528692127868 kJ/mol
Potential energy after minimization is 5.295784483017486 kJ/mol
Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
250,35.3293610728431,377.99360963724604
500,32.914370795078696,266.32383093566114
750,54.91832565536845,338.4975975582279
1000,31.951928630267396,406.43819755911716
1250,53.82859327570476,369.464024840027
1500,59.823407955749836,197.6425679517721
1750,56.30366278285174,307.609001207952
2000,49.92505130993653,485.10406481983694
2250,14.968692250930108,300.55778667777446
2500,29.001963693973394,294.58025418518196
2750,47.887653179717425,325.61043102430165
3000,48.788049158950486,133.6404008212833
3250,50.483163386784874,266.1492111159933
3500,43.46946634541573,266.6659552455204
3750,44.108708594154514,409.8434146285466
4000,33.02809649744729,317.9246768033946
4250,37.129607070303514,440.31910053582794
4500,29.39123231117899,384.8577492390589
4750,42.53260375254895,211.37590920

---

## Analysis
Now that we’ve performed our computer experiment, it is time to analyze the data we have collected. The main type of data you have collected through this simulation is information on atom positions, or the system trajectory.

As part of our production simulation, we set up a reporter to record atomic positions. The code below shows that code from your previous script, **you do not need to execute it.**

In [ ]:
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

This reporter saved the atomic positions for us every 100 timesteps in a file called `ethane_sim.dcd`. The DCD file format is a binary file (instead of being a text file), so you cannot open it and look at it. However, we will be using certain libraries to analyze and view the file’s contents. If you’ve run your simulation, you should have the file `ethane_sim.dcd` in the same folder as your Jupyter notebook.

First, we will need to make sure we have a few more Python libraries installed which can help us with analysis. We will use a library called **nglview** to visualize the trajectory, and a library called **MDTraj** to analyze the trajectory. Before opening a new notebook for analysis, you may need to install **nglview** and **MDTraj**.

Type the following in your *terminal* to install **nglview** and **MDTraj**:

In [ ]:
$ conda install -c conda-forge mdtraj nglview

In [ ]:
import mdtraj as md

traj = md.load('ethane_sim.dcd', top='data/ethane.pdb')

The command above reads all of the atomic positions from `ethane_sim.dcd` and keeps track of atom connectivity (topology) which was given in the PDB file. Next, visualize the trajectory using nglview. Nglview has a special function `show_mdtraj` that we can use with our trajectory because it was in a specific format from the MDTraj library.

In [ ]:
import nglview as ngl

visualize = ngl.show_mdtraj(traj)
visualize

This should show you something that looks sort of like a movie of your ethane molecule. These are the atomic positions calculated by OpenMM during the molecular dynamics run. We can now analyze the positions to find out some things about our molecule.

We will use another OpenMM command to pull out our bonds and atoms for analysis

In [ ]:
atoms, bonds = traj.topology.to_dataframe()
atoms

#### QUESTION
Describe the table that `atoms` produced.

#### RESPONSE
Your response here :D

### Analyzing the C-C bond length
Let’s look at what C-C bond lengths our ethane molecule had during the simulation. Before we can measure the bond lengths, we have to decide which atoms from our molecule define the bond angle. You should see a table output from running `atoms`.

We have to pick the atom indices for the C-C bond. An atom’s index is the left-most value in the table above. For our torsion, we’ll measure `C1-C2` the indices for these are `0` and `4`. We use the function `compute_distances` in the MDTraj library to measure the distance between these atoms.

In [ ]:
bond_indices = [0, 4] # atoms to define the bond length
bond_length = md.compute_distances(traj, [bond_indices])

We now have the measurement for this torsion angle in radians for each recorded timestep of the trajectory saved in the array `bond_length`. One way we can examine this data is by plotting it as a histogram using the Python library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

bondcounts, binedges, otherstuff = plt.hist(bond_length, bins=120)
plt.title('C-C bond length histogram')
plt.xlabel('Bond length (nm)')
plt.ylabel('Counts')
plt.show()

## Your Turn
A torsion is made up of four atoms which are bonded to each other. Analyze the torsion angle associated with the atoms `H11-C1-C2-H21` for your trajectory. Instead of using the function `compute_distance`, use `compute_dihedrals`. Create a histogram plot of the torsion angles.

First, we need to pick the atom indices of our torsion angle and use the `compute_dihedrals` function to calculate the dihedrals.

In [ ]:
phi_indices = [] # add atoms to define the torsion angle
phi = # use the compute_dihedrals function

print(phi)

We now have the measurement for this torsion angle in radians for each recorded timestep of the trajectory.

Next, we can examine this data by plotting it as a histogram using the Python library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

# create a histogram with 90 bins

Credit for this exercise goes to [MolSSI](https://education.molssi.org/mm-tools/02-md-alkanes/index.html).